In [1]:
import os
import json
from web3 import Web3

#  compile  solidity
# https://github.com/iamdefinitelyahuman/py-solc-x
from solcx import compile_standard, install_solc

with open('./contracts/IANA.sol', 'r', encoding='utf-8') as f:
    iana_file = f.read()

#  download 0.8.0 Version of Solidity compiler 
install_solc('0.8.0')

#  compile Solidity
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        # Solidity file 
        "sources": {"IANA.sol": {"content": iana_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    #  Content generated after compilation 
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    #  edition , When writing smart contracts Solidity The version used corresponds to 
    solc_version="0.8.0",
)

#  The compiled results are written to the file 
with open('compiled_code.json', 'w') as f:
    json.dump(compiled_sol, f)

In [2]:
from compile import *
from dotenv import load_dotenv

load_dotenv()

w3 = Web3(Web3.HTTPProvider(os.getenv("GANACHE_RPC_URL")))
chain_id = 1337

my_address = os.getenv("ACCOUNT_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")

#  Compiled bytecode of smart contract （ Data on the chain ）
bytecode = compiled_sol["contracts"]["IANA.sol"]["IANA"]["evm"]["bytecode"]["object"]

# ABI (Application Binary Interface), An interface for interacting with methods in a smart contract 
abi = json.loads(
    compiled_sol["contracts"]["IANA.sol"]["IANA"]["metadata"]
    )["output"]["abi"]

#  Build smart contract objects 
iana = w3.eth.contract(abi=abi, bytecode=bytecode)
#  Of the last transaction in the current blockchain nonce
nonce = w3.eth.get_transaction_count(my_address)

#  Deploy smart contracts  -  Create transaction 
transaction = iana.constructor().buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id, 
    "from": my_address, 
    "nonce": nonce
    }
)
#  Sign the current transaction  -  Prove that you initiated the transaction 
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
print("Deploying Contract!")

#  Start deployment  -  Sending transaction 
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print('Waiting for deploy transaction to finish...')
#  Wait for smart contract deployment results , After deployment , Will get the address of the contract 
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print('Deployed Done!')
print("contract address: " + tx_receipt.contractAddress)

Deploying Contract!
Waiting for deploy transaction to finish...
Deployed Done!
contract address: 0x6070e7461273f681c895C9eC9B9Cec25aA232A11


In [14]:
with open('./contracts/Recover.sol', 'r', encoding='utf-8') as f:
    recover_file = f.read()

#  download 0.8.0 Version of Solidity compiler 
install_solc('0.8.0')

#  compile Solidity
compiled_recover_sol = compile_standard(
    {
        "language": "Solidity",
        # Solidity file 
        "sources": {"Recover.sol": {"content": recover_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    #  Content generated after compilation 
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    #  edition , When writing smart contracts Solidity The version used corresponds to 
    solc_version="0.8.0",
)

#  The compiled results are written to the file 
with open('compiled_recover_code.json', 'w') as f:
    json.dump(compiled_recover_sol, f)

In [15]:
from compile_recover import *
from dotenv import load_dotenv

load_dotenv()

w3 = Web3(Web3.HTTPProvider(os.getenv("GANACHE_RPC_URL")))
chain_id = 1337

my_address = os.getenv("ACCOUNT_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")

#  Compiled bytecode of smart contract （ Data on the chain ）
bytecode = compiled_recover_sol["contracts"]["Recover.sol"]["Recover"]["evm"]["bytecode"]["object"]

# ABI (Application Binary Interface), An interface for interacting with methods in a smart contract 
abi = json.loads(
    compiled_recover_sol["contracts"]["Recover.sol"]["Recover"]["metadata"]
    )["output"]["abi"]

#  Build smart contract objects 
recover = w3.eth.contract(abi=abi, bytecode=bytecode)
#  Of the last transaction in the current blockchain nonce
nonce = w3.eth.get_transaction_count(my_address)

#  Deploy smart contracts  -  Create transaction 
transaction = recover.constructor().buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id, 
    "from": my_address, 
    "nonce": nonce
    }
)
#  Sign the current transaction  -  Prove that you initiated the transaction 
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
print("Deploying Contract!")

#  Start deployment  -  Sending transaction 
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print('Waiting for deploy transaction to finish...')
#  Wait for smart contract deployment results , After deployment , Will get the address of the contract 
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print('Deployed Done!')
print("contract address: " + tx_receipt.contractAddress)

Deploying Contract!
Waiting for deploy transaction to finish...
Deployed Done!
contract address: 0x4Baaf160c88e5B03AC53Ec471E22fb5Ae7935d9A


In [74]:
from utils.utils import *
from eth_account.messages import encode_defunct, _hash_eip191_message

In [7]:
load_dotenv()

w3 = Web3(Web3.HTTPProvider(os.getenv("GANACHE_RPC_URL")))
chain_id = 1337
my_address = os.getenv("ACCOUNT_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")
# ABI (Application Binary Interface), An interface for interacting with methods in a smart contract 
abi = json.loads(
    compiled_sol["contracts"]["IANA.sol"]["IANA"]["metadata"]
    )["output"]["abi"]

#  call deploy.py Will get contract_address
contract_address = os.getenv("CONTRACT_ADDRESS")

nonce = w3.eth.get_transaction_count(my_address)

#  Instantiate the contract object 
iana = w3.eth.contract(address=contract_address, abi=abi)

In [68]:
msgToBeSigned = iana.functions.IANA_getSignatureMessage(2, os.getenv("ACCOUNT2_ADDRESS")).call()
print(msgToBeSigned)
msgToBeSignedEncoded = encode_defunct(text=msgToBeSigned.hex()) 	# encode msg
print(msgToBeSignedEncoded)

ValueError: Please ensure that this contract instance has an address.

In [30]:
acct2_priv_key = os.getenv("ACCOUNT2_PRIVATE_KEY")

In [31]:
msgSigned = w3.eth.account.sign_message(msgToBeSignedEncoded, private_key=acct2_priv_key) # sign message
print(msgSigned)

SignedMessage(messageHash=HexBytes('0xc46ca3f13027eab29a2708ee0a92a336626483d0a8ae661eeefc37e72ceb8b4e'), r=25220368283057574198236257725980923427714442422506267022753236144755710097102, s=53081634886786488305536368506385894731144849654380448965729957872795793693582, v=27, signature=HexBytes('0x37c238fb94c8f18ce8b6ff4276dc8a3fbab8a5f2f085ebcfdc0d539af18ff6ce755b24021b2af92f481af1f4d91bf5ce5851951ed415466985a9600f122f1b8e1b'))


In [32]:
# Check that we can recover the message we signed.
# Output should be the public address of ACCOUNT2_ADDRESS
addr = w3.eth.account.recover_message(msgToBeSignedEncoded, signature=msgSigned.signature) # output should be public address of acct2
print(addr)
print(addr == os.getenv("ACCOUNT2_ADDRESS"))

0x79bb7739B28ab9D6a846012156f5eadD0dE67361
True


In [36]:
# Generate sigV, sigR, sigS for IANA validation
ec_recover_args_iana = (msgSignedHash, sigV, sigR, sigS) = (Web3.toHex(msgSigned.messageHash), msgSigned.v, utils.to_32byte_hex(msgSigned.r), utils.to_32byte_hex(msgSigned.s))
print(ec_recover_args_iana)

('0xc46ca3f13027eab29a2708ee0a92a336626483d0a8ae661eeefc37e72ceb8b4e', 27, '0x37c238fb94c8f18ce8b6ff4276dc8a3fbab8a5f2f085ebcfdc0d539af18ff6ce', '0x755b24021b2af92f481af1f4d91bf5ce5851951ed415466985a9600f122f1b8e')


In [65]:
load_dotenv(override=True)

my_address = os.getenv("ACCOUNT_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")
# ABI (Application Binary Interface), An interface for interacting with methods in a smart contract 
abi = json.loads(
    compiled_recover_sol["contracts"]["Recover.sol"]["Recover"]["metadata"]
    )["output"]["abi"]

#  call deploy.py Will get contract_address
contract_address = os.getenv("RECOVER_CONTRACT_ADDRESS")
print(contract_address)
nonce = w3.eth.get_transaction_count(my_address)

#  Instantiate the contract object 
recover = w3.eth.contract(address=contract_address, abi=abi)

0x5947F040323EA9137c51daCF3A4E6215563d901a


In [72]:
# Check we can recover via Recover contract -> output should be acct2.address

tx = recover.functions.ecr(msgSignedHash, sigV, sigR, sigS).call()
print(tx)


0x79bb7739B28ab9D6a846012156f5eadD0dE67361


In [4]:

from utils.utils import *
from eth_account.messages import encode_defunct, _hash_eip191_message

inASN = 13
inAddress = Web3.toChecksumAddress("0x79bb7739B28ab9D6a846012156f5eadD0dE67361")
print(inAddress)
# inAddress = int(sys.argv[2], 16)
# inAddress = hex(inAddress)

load_dotenv(override=True)

w3 = Web3(Web3.HTTPProvider(os.getenv("GANACHE_RPC_URL")))
chain_id = 1337

my_address = os.getenv("ACCOUNT_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")

acct2_private_key = os.getenv("ACCOUNT2_PRIVATE_KEY")

# ABI (Application Binary Interface), An interface for interacting with methods in a smart contract 
abi = json.loads(
    compiled_sol["contracts"]["IANA.sol"]["IANA"]["metadata"]
    )["output"]["abi"]

#  call deploy.py Will get contract_address
contract_address = os.getenv("CONTRACT_ADDRESS")

nonce = w3.eth.get_transaction_count(my_address)

#  Instantiate the contract object 
iana = w3.eth.contract(address=contract_address, abi=abi)

msgToBeSigned = iana.functions.IANA_getSignatureMessage(13, inAddress).call()
msgToBeSignedEncoded = utils.encode_defunct_wrapper(msgToBeSigned) 	# encode msg
msgSigned = utils.sign_message_wrapper(w3, msgToBeSignedEncoded, acct2_private_key) # sign message
msgSignedHash, sigV, sigR, sigS = utils.generate_sig_v_r_s(msgSigned) # generate hash of signed msg, sigV, sigR, sigS

#  call addPrefix Method 
transaction = iana.functions.IANA_addASNSigned(inASN, inAddress, msgSignedHash, sigV, sigR, sigS).buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id,
    "from": my_address,
    "nonce": nonce
})
#  Signature 
signed_transaction = w3.eth.account.sign_transaction(transaction, private_key=private_key)
#  Sending transaction 
tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
print('add new ASN to ASN Map SIGNED...')
#  Waiting for the deal to complete 
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("ASN SIGNED added")

0x79bb7739B28ab9D6a846012156f5eadD0dE67361


ContractLogicError: execution reverted: VM Exception while processing transaction: revert ASN<=>ASNOwner mapping already added

In [9]:
from ipaddress import IPv4Address

inASN = 13
inIP = IPv4Address("10.0.20.0")
inSubnet = 24
inAsnAddress = Web3.toChecksumAddress("0xcF0A72EFd623c7aC7f9886213daFd93a2D628327")

load_dotenv()

w3 = Web3(Web3.HTTPProvider(os.getenv("GANACHE_RPC_URL")))
chain_id = 1337

my_address = os.getenv("ACCOUNT_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")

# acct2_address = os.getenv("ACCOUNT2_ADDRESS")
acct2_private_key = os.getenv("ACCOUNT2_PRIVATE_KEY") # must be privkey of inAsnAddress

# ABI (Application Binary Interface), An interface for interacting with methods in a smart contract 
abi = json.loads(
    compiled_sol["contracts"]["IANA.sol"]["IANA"]["metadata"]
    )["output"]["abi"]

#  call deploy.py Will get contract_address
contract_address = os.getenv("CONTRACT_ADDRESS")

nonce = w3.eth.get_transaction_count(my_address)

#  Instantiate the contract object 
iana = w3.eth.contract(address=contract_address, abi=abi)

print("passed in asn address: " + str(inAsnAddress))

msgToBeSigned = iana.functions.IANA_getPrefixSignatureMessage(int(inIP), inSubnet, inASN, inAsnAddress).call()
msgToBeSignedEncoded = utils.encode_defunct_wrapper(msgToBeSigned) 	# encode msg
msgSigned = utils.sign_message_wrapper(w3, msgToBeSignedEncoded, acct2_private_key) # sign message
msgSignedHash, sigV, sigR, sigS = utils.generate_sig_v_r_s(msgSigned) # generate hash of signed msg, sigV, sigR, sigS




passed in asn address: 0xcF0A72EFd623c7aC7f9886213daFd93a2D628327


In [13]:
print(inAsnAddress, inASN)

0xcF0A72EFd623c7aC7f9886213daFd93a2D628327 13


In [17]:
addr = w3.eth.account.recover_message(msgToBeSignedEncoded, signature=msgSigned.signature)
print(addr)

0x79bb7739B28ab9D6a846012156f5eadD0dE67361
